# Reconciling Object & Organisation Types


In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import os

from heritageconnector.config import config
from heritageconnector.utils.data_transformation import transform_series_str_to_list
from heritageconnector.entity_matching.reconciler import reconciler

from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Organisations

In [2]:
df = pd.read_pickle("../GITIGNORE_DATA/results/filtering_people_orgs_result.pkl")
# df_people = df[df['GENDER'].isin(["M", "F"])]
df_orgs = df[df['GENDER'] == "N"]

In [3]:
org_type_col = "OCCUPATION"

In [4]:
df_orgs.loc[:, org_type_col] = df_orgs.loc[:, org_type_col].str.replace("'", "")
df_orgs.loc[:, org_type_col] = transform_series_str_to_list(df_orgs[org_type_col], separator=";")

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [5]:
rec = reconciler(df_orgs, table="ORGANISATION")
df_orgs["OCCUPATION_resolved"] = rec.process_column(org_type_col,
                                                    multiple_vals=True, 
                                                    class_include=["Q43229", "Q28640"], 
                                                    text_similarity_thresh=95,
                                                    search_limit_per_item=500)

2020-10-02 16:28:22,944 - heritageconnector.entity_matching.reconciler - INFO - Looking up Wikidata qcodes for items on Elasticsearch Wikidata dump


100%|██████████| 2603/2603 [06:02<00:00,  7.17it/s]

2020-10-02 16:34:25,878 - heritageconnector.entity_matching.reconciler - INFO - Filtering to values in subclass tree of ['Q43229', 'Q28640']



100%|██████████| 58/58 [00:34<00:00,  1.69it/s]
/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
#df_orgs[[org_type_col, "OCCUPATION_resolved"]]#.head()
print(f"{int((df_orgs['OCCUPATION_resolved'].apply(len) > 0).sum() / len(df_orgs) * 100)}% of records have at least one resolved type")

24% of records have at least one resolved type


In [7]:
df_orgs.to_pickle("./df_orgs.pkl")

## Objects

In [8]:
objects = pd.read_csv("../GITIGNORE_DATA/smg-datasets-private/mimsy-catalogue-export.csv")

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,5,6,8,12,18,25,26,27,30,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
objects.loc[:, "ITEM_NAME"] = transform_series_str_to_list(objects["ITEM_NAME"], separator=";")

In [ ]:
rec_o = reconciler(objects, table="OBJECT")
# physical object
objects["ITEM_NAME_resolved"] = rec_o.process_column("ITEM_NAME", multiple_vals=True, class_include="Q223557", class_exclude=["Q5", "Q43229", "Q28640"], text_similarity_thresh=95)

In [ ]:
objects[['ITEM_NAME', 'ITEM_NAME_resolved']]

In [ ]:
print(f"{int((objects['ITEM_NAME_resolved'].apply(len) > 0).sum() / len(objects) * 100)}% of records have at least one resolved type")